In [1]:
# import modules
import discord
import os
from neuralintents.assistants import BasicAssistant

# from neuralintents import BasicAssistant
import nest_asyncio
import requests
import json
import random

print(discord.__file__)

nest_asyncio.apply()

TOKEN = "MTE2NjYzNTczNjA5NTc5NzMyMQ.G2hPjM.PvTVKR7uoJWoAHRHjG5P6iS82FzJdGy4Z21qDg"
client = discord.Client(intents=discord.Intents.all())

c:\Users\MrChe\AppData\Local\Programs\Python\Python311\Lib\site-packages\discord\__init__.py


In [2]:
def get_quote():
    response = requests.get("https://zenquotes.io/api/random")
    json_data = json.loads(response.text)
    quote = json_data[0]["q"] + " -" + json_data[0]["a"]
    return quote

    # ==================================================================================================== Test


# def print_stocks():
# return "Here are the stocks I found: "

chatbot = BasicAssistant("intents.json")

chatbot.fit_model(epochs=50)
chatbot.save_model()


# ==================================================================================================== Test


class MyClient(discord.Client):
    async def on_ready(self):
        print("---------------------------------------------------")
        print(f"Logged in as {self.user} (ID: {self.user.id})")
        print("---------------------------------------------------")
        print("")

    async def on_message(self, message):
        # we do not want the bot to reply to itself
        if message.author.id == self.user.id:
            return

        # while the bot is waiting on a response from the model
        # set the its status as typing for user-friendliness
        async with message.channel.typing():
            if message.content.startswith("!hello"):
                await message.reply("Hello!", mention_author=True)

            # THE ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ AI!
            if message.content.startswith(""):  # no prefix
                response = chatbot.process_input(message.content)
                await message.channel.send(response)


intents = discord.Intents.default()
intents.message_content = True


def main():
    client = MyClient(intents=intents)
    client.run(TOKEN)


if __name__ == "__main__":
    main()

Epoch 1/50
105/105 [==============================] - 1s 1ms/step - loss: 4.5665 - accuracy: 0.0345
Epoch 2/50
105/105 [==============================] - 0s 2ms/step - loss: 4.0421 - accuracy: 0.1264
Epoch 3/50
105/105 [==============================] - 0s 2ms/step - loss: 3.5757 - accuracy: 0.2222
Epoch 4/50
105/105 [==============================] - 0s 2ms/step - loss: 3.3572 - accuracy: 0.2299
Epoch 5/50
105/105 [==============================] - 0s 2ms/step - loss: 3.1864 - accuracy: 0.2835
Epoch 6/50
105/105 [==============================] - 0s 2ms/step - loss: 2.8417 - accuracy: 0.3257
Epoch 7/50
105/105 [==============================] - 0s 1ms/step - loss: 2.7177 - accuracy: 0.3621
Epoch 8/50
105/105 [==============================] - 0s 1ms/step - loss: 2.6989 - accuracy: 0.3697
Epoch 9/50
105/105 [==============================] - 0s 1ms/step - loss: 2.6161 - accuracy: 0.3678
Epoch 10/50
105/105 [==============================] - 0s 2ms/step - loss: 2.5228 - accuracy: 0.4253

[2023-11-15 20:34:07] [INFO    ] discord.client: logging in using static token
[2023-11-15 20:34:08] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: e955f78da91149e5562dbd6ad5314192).


---------------------------------------------------
Logged in as Snowman#0193 (ID: 1166635736095797321)
---------------------------------------------------

